Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
num_steps = 3001

def run_graph(graph):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 1000 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [6]:
# l2_norm to the logistic model
batch_size = 128

logistic_graph = tf.Graph()
with logistic_graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # add small constant for l2 loss
    beta_regularizer = tf.constant(1e-3, tf.float32)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # add l2 loss
    loss += beta_regularizer * tf.nn.l2_loss(weights)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
run_graph(logistic_graph)

Initialized
Minibatch loss at step 0: 22.504890
Minibatch accuracy: 7.0%
Validation accuracy: 11.9%
Minibatch loss at step 1000: 1.520997
Minibatch accuracy: 80.5%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 0.958696
Minibatch accuracy: 78.9%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 1.017530
Minibatch accuracy: 78.9%
Validation accuracy: 80.7%
Test accuracy: 87.3%


In [7]:
# l2_norm to the non-linear model
NUM_HIDDEN_NODES = 1024

nonlinear_graph = tf.Graph()
with nonlinear_graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # add small constant for l2 loss
    beta_regularizer = tf.constant(1e-3, tf.float32)

    # Variables.
    # add more weights and biases
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, NUM_HIDDEN_NODES]))
    bias1 = tf.Variable(tf.zeros([NUM_HIDDEN_NODES]))
    weights2 = tf.Variable(tf.truncated_normal([NUM_HIDDEN_NODES, num_labels]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # add non-linearity after first set of weights
    def get_logits(dataset):
        layer1 = tf.nn.relu(tf.matmul(dataset, weights1) + bias1)
        return tf.matmul(layer1, weights2) + bias2
    logits = get_logits(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # add l2 loss
    loss += beta_regularizer * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # add valid and test layers
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(get_logits(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_logits(tf_test_dataset))
run_graph(nonlinear_graph)

Initialized
Minibatch loss at step 0: 686.199463
Minibatch accuracy: 6.2%
Validation accuracy: 40.9%
Minibatch loss at step 1000: 114.311211
Minibatch accuracy: 84.4%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 41.272243
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 15.507002
Minibatch accuracy: 82.0%
Validation accuracy: 87.2%
Test accuracy: 93.0%


The regularization is very effective.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
num_steps = 3001
NUM_BATCHES = 10

with tf.Session(graph=nonlinear_graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = step % NUM_BATCHES
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 658.516418
Minibatch accuracy: 6.2%
Validation accuracy: 30.3%
Minibatch loss at step 1000: 115.952164
Minibatch accuracy: 100.0%
Validation accuracy: 69.0%
Minibatch loss at step 2000: 42.645767
Minibatch accuracy: 100.0%
Validation accuracy: 68.8%
Minibatch loss at step 3000: 15.684834
Minibatch accuracy: 100.0%
Validation accuracy: 69.0%
Test accuracy: 76.4%


With only little data the model fits the training data quickly, but generalises poorly. It's overfitting the training data.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [9]:
def run_limited_batches_with_dropout(graph):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            offset = step % NUM_BATCHES
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, keep_prob: 0.5}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 1000 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % 
                      accuracy(session.run(valid_prediction, feed_dict={keep_prob: 1.0}), valid_labels))
        print("Test accuracy: %.1f%%" % 
              accuracy(session.run(test_prediction, feed_dict={keep_prob: 1.0}), test_labels))

In [10]:
# l2_norm to the non-linear model with dropout
nonlinear_dropout_graph = tf.Graph()
with nonlinear_dropout_graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regularizer = tf.constant(1e-3, tf.float32)
    keep_prob = tf.placeholder(tf.float32)

    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, NUM_HIDDEN_NODES]))
    bias1 = tf.Variable(tf.zeros([NUM_HIDDEN_NODES]))
    weights2 = tf.Variable(tf.truncated_normal([NUM_HIDDEN_NODES, num_labels]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    def get_logits(dataset):
        layer1 = tf.nn.relu(tf.matmul(dataset, weights1) + bias1)
        # add dropout
        layer1 = tf.nn.dropout(layer1, keep_prob)
        return tf.matmul(layer1, weights2) + bias2
    logits = get_logits(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta_regularizer * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(get_logits(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_logits(tf_test_dataset))

run_limited_batches_with_dropout(nonlinear_dropout_graph)

Initialized
Minibatch loss at step 0: 792.370972
Minibatch accuracy: 8.6%
Validation accuracy: 30.9%
Minibatch loss at step 1000: 116.126030
Minibatch accuracy: 100.0%
Validation accuracy: 71.4%
Minibatch loss at step 2000: 42.713902
Minibatch accuracy: 100.0%
Validation accuracy: 71.0%
Minibatch loss at step 3000: 15.709908
Minibatch accuracy: 100.0%
Validation accuracy: 71.3%
Test accuracy: 78.8%


The dropout had a positive effect on the overfitting.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [11]:
num_steps = 5001

def run_dropout_graph(graph):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, keep_prob: 0.5}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 1000 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % 
                      accuracy(session.run(valid_prediction, feed_dict={keep_prob: 1.0}), valid_labels))
        print("Test accuracy: %.1f%%" % 
              accuracy(session.run(test_prediction, feed_dict={keep_prob: 1.0}), test_labels))

In [12]:
NUM_HIDDEN1 = 1024
NUM_HIDDEN2 = 512
NUM_HIDDEN3 = 246

final_graph = tf.Graph()
with final_graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset, tf_test_dataset = tf.constant(valid_dataset), tf.constant(test_dataset)
    
    #beta_regularizer = tf.constant(1e-4, tf.float32)
    keep_prob = tf.placeholder(tf.float32)

    # Variables.
    W1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, NUM_HIDDEN1],
                            stddev=0.01))
    b1 = tf.Variable(tf.zeros([NUM_HIDDEN1]))
    W2 = tf.Variable(
        tf.truncated_normal([NUM_HIDDEN1, NUM_HIDDEN2],
                            stddev=0.01))
    b2 = tf.Variable(tf.zeros([NUM_HIDDEN2]))
    W3 = tf.Variable(
        tf.truncated_normal([NUM_HIDDEN2, NUM_HIDDEN3],
                            stddev=0.01))
    b3 = tf.Variable(tf.zeros([NUM_HIDDEN3]))
    
    final_W = tf.Variable(
        tf.truncated_normal([NUM_HIDDEN3, num_labels], 
                            stddev=0.01))
    final_b = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    def get_logits(dataset):
        layer = tf.nn.relu(tf.matmul(dataset, W1) + b1)
        layer = tf.nn.dropout(layer, keep_prob)
        layer = tf.nn.relu(tf.matmul(layer, W2) + b2)
        layer = tf.nn.dropout(layer, keep_prob)
        layer = tf.nn.relu(tf.matmul(layer, W3) + b3)
        return tf.matmul(layer, final_W) + final_b
    logits = get_logits(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    #loss += beta_regularizer * (np.sum(tf.nn.l2_loss(W) for W in [W1, W2, W3, final_W]))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(get_logits(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_logits(tf_test_dataset))
run_dropout_graph(final_graph)

Initialized
Minibatch loss at step 0: 2.302522
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.406966
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 2000: 0.424159
Minibatch accuracy: 86.7%
Validation accuracy: 87.3%
Minibatch loss at step 3000: 0.551296
Minibatch accuracy: 80.5%
Validation accuracy: 87.8%
Minibatch loss at step 4000: 0.285228
Minibatch accuracy: 89.8%
Validation accuracy: 88.7%
Minibatch loss at step 5000: 0.161287
Minibatch accuracy: 96.1%
Validation accuracy: 88.8%
Test accuracy: 94.1%


**Note to self:** 
If the standard deviation of the weights was not set properly, the training would quickly return NaN for loss.
Probably some vanishing gradients of something similar.
The L2 regularization did not have much effect while dropout was used.